In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# Read the contents of the file
file = Path('WhatsApp Chat with Pippy 😈.txt')
with file.open('r') as f:
    raw_text = f.read()

In [ ]:
# Parse the contents of the file
from delicious_treat.parser import parse
data = parse(raw_text)
data

In [ ]:
pippy_messages = data[data.name == 'Pippy 😈'].drop(columns=['name'])
pippy_messages

# Simple counts

In [ ]:
from delicious_treat.analyser import Analyser
# Create an analyser for Pippy's messages
analyser = Analyser(pippy_messages)
analyser.analyse()

In [ ]:
fd = analyser.freq_dist(pos=True)

In [ ]:
fd.conditions()

In [ ]:
fd[''].most_common(40)

In [ ]:
analyser.filter_messages('sex')

In [ ]:
import math
from datetime import datetime
import matplotlib.dates as mdate

def get_date_as_datetime(dt):
    return datetime.combine(dt.date(), datetime.min.time())

def get_bins(times, duration):
    first_date = get_date_as_datetime(times.min())
    total_seconds = (get_date_as_datetime(times.max()) - first_date).total_seconds()
    bin_count = math.ceil(total_seconds / duration.total_seconds())
    return [mdate.epoch2num(datetime.timestamp(first_date + step * duration)) for step in range(bin_count)]

In [ ]:
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

fig, axes = plt.subplots(2, 1)

# Get bins from all the messages
bins = get_bins(analyser.messages.time, timedelta(days=1))

# Plot a filtered subset
total, _, _ = axes[0].hist(analyser.messages.time, bins=bins)
sub, _, _ = axes[0].hist(analyser.filter_messages('sex').time, bins=bins)

# Plot scaled
axes[1].bar(bins[:-1], sub / total * 100)